In [1]:
import pandas as pd

def read_ply_ascii(filepath):
    """
    Reads an ASCII PLY file and returns the header lines and a pandas DataFrame
    of the vertex data.
    """
    header = []
    with open(filepath, 'r') as f:
        # Read header until 'end_header'
        for line in f:
            header.append(line.rstrip())
            if line.strip() == 'end_header':
                break
        
        # The remaining lines are the data
        data = [list(map(float, line.split())) for line in f if line.strip()]
    
    # Create DataFrame
    df = pd.DataFrame(data, columns=['x', 'y', 'z', 'scalar_Classification'])
    return header, df


def process_classifications(df):
    """
    Removes unwanted classifications and applies remapping rules.
    Returns filtered + remapped DataFrame and unique value sets before and after.
    """
    # Distinct before
    unique_before = sorted(df['scalar_Classification'].unique())

    # Remove unwanted classifications
    remove_classes = {0.0, 7.0, 9.0, 55.0}
    df = df[~df['scalar_Classification'].isin(remove_classes)].copy()

    # Apply mapping
    mapping = {
        1.0: 0.0,
        2.0: 2.0, 3.0: 2.0,
        4.0: 8.0, 6.0: 8.0,
        5.0: 1.0
    }
    df['scalar_Classification'] = df['scalar_Classification'].replace(mapping)

    # Distinct after
    unique_after = sorted(df['scalar_Classification'].unique())
    return df, unique_before, unique_after


def write_ply_ascii(filepath, header, df):
    """
    Writes the filtered and modified point cloud back to an ASCII PLY file.
    Updates vertex count in the header.
    """
    # Update vertex count line
    new_header = []
    for line in header:
        if line.startswith('element vertex'):
            new_header.append(f'element vertex {len(df)}')
        else:
            new_header.append(line)
    
    # Write back to file
    with open(filepath, 'w') as f:
        for line in new_header:
            f.write(line + '\n')
        for _, row in df.iterrows():
            f.write(f"{row.x:.6f} {row.y:.6f} {row.z:.6f} {row.scalar_Classification:.6f}\n")


def main():
    input_ply = 'IIITB_Base.ply'   # Change this to your file path
    output_ply = 'IIITB_Base_train.ply'

    header, df = read_ply_ascii(input_ply)
    processed_df, unique_before, unique_after = process_classifications(df)

    print("Unique classification values BEFORE processing:")
    print(unique_before)
    print("\nUnique classification values AFTER processing:")
    print(unique_after)

    write_ply_ascii(output_ply, header, processed_df)
    print(f"\nFiltered PLY written to: {output_ply}")
    print(f"Remaining points: {len(processed_df)}")


if __name__ == "__main__":
    main()


Unique classification values BEFORE processing:
[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 9.0, 55.0]

Unique classification values AFTER processing:
[0.0, 1.0, 2.0, 8.0]

Filtered PLY written to: IIITB_Base_train.ply
Remaining points: 2179723


In [3]:
def convert_class_field(input_file, output_file):
    with open(input_file, 'r') as infile:
        lines = infile.readlines()

    # Identify the header and the start of vertex data
    header_end = lines.index('end_header\n') + 1
    vertex_lines = lines[header_end:]

    # Update the class field (4th field in each vertex line)
    updated_vertex_lines = []
    for line in vertex_lines:
        # Split the line into components (x, y, z, class)
        parts = line.split()
        if len(parts) >= 4:
            # Convert the class field to int
            parts[3] = str(int(float(parts[3])))  # Convert class field from float to int
        # Join the parts back into a single line
        updated_vertex_lines.append(" ".join(parts))
        updated_vertex_lines.append("\n")

    # Write the updated content back to a new file
    with open(output_file, 'w') as outfile:
        # Write the header and updated vertex data
        outfile.writelines(lines[:header_end])  # Write header
        outfile.writelines(updated_vertex_lines)  # Write updated vertex lines

# Usage
input_file = 'IIITB_Base_train.ply'  # Replace with your input PLY file path
output_file = 'IIITB_Base_train2.ply' # Replace with the desired output PLY file path

convert_class_field(input_file, output_file)
